In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data/titanic_train.csv')
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


average age by gender

In [4]:
df.groupby('Sex')['Age'].mean()

Sex
female    27.915709
male      30.726645
Name: Age, dtype: float64

In [5]:
df.groupby('Sex')['Survived'].sum()

Sex
female    233
male      109
Name: Survived, dtype: int64

survived by pclass and sex

In [10]:
df.groupby(['Pclass','Sex'])['Survived'].sum().to_frame('Survived count')

Pclass  Sex   
1       female    91
        male      45
2       female    70
        male      17
3       female    72
        male      47
Name: Survived, dtype: int64

People above 30 who survived, by sex

In [11]:
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [12]:
df.groupby('Sex').apply(lambda subdf: subdf[(subdf.Age > 30) & (subdf.Survived == 1)].shape[0])

Sex
female    83
male      41
dtype: int64

In [20]:
df.head(2)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C


In [28]:
idx = pd.IndexSlice

In [31]:
is_pc = df['Ticket'].str.split().str.get(0) == 'PC'
df.loc[is_pc,'Pclass'].value_counts()

1    60
Name: Pclass, dtype: int64

In [45]:
(df
 .groupby(
     df['Ticket'].str.split().str.get(0) #groupby the df by the first values after spliting the Ticket column by a whitespace
 )
 ['Pclass'] # accessing the Pclass column
 .apply(lambda x: x.value_counts())
 .loc[idx[:,3]]
)

Ticket
12460        1
14311        1
14312        1
14313        1
14973        1
            ..
SOTON/O2     2
SOTON/OQ     7
STON/O      12
STON/O2.     6
W./C.        7
Name: Pclass, Length: 334, dtype: int64

example for W/ c

In [33]:
wc_mask = df['Ticket'].str.split().str.get(0) == "W./C."
wc_mask.value_counts()

False    882
True       9
Name: Ticket, dtype: int64

In [35]:
df.loc[wc_mask,'Pclass'].value_counts()

3    7
2    2
Name: Pclass, dtype: int64

how to better fill the nan age rows

In [38]:
df.head(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S


In [51]:
def baseline_age_imputer(df):
    df = df.copy()
    df.loc[:,'Age'] = df.Age.fillna(df['Age'].median())
    return df

def age_imputer_using_Sex(df):
    df = df.copy()
    median_age_by_sex = df.groupby('Sex').Age.median()
    
    for gender in median_age_by_sex.index:
        df.loc[(df['Sex'] == gender) & (df.Age.isna()),'Age'] = median_age_by_sex[gender]
    return df

#baseline_age_imputer(df)
age_imputer_using_Sex(df)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,27.0,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C
